In [ ]:
import tensorflow as tf
import numpy as np
import os
tf.reset_default_graph()

flags = tf.app.flags
flags.DEFINE_string('data_dir', '/work/cse496dl/shared/hackathon/02/mnist/', 'directory where MNIST is located')
flags.DEFINE_string('save_dir', 'hackathon_3', 'directory where model graph and weights are saved')
flags.DEFINE_integer('batch_size', 50, '')
flags.DEFINE_integer('max_epoch_num', 100, '')
FLAGS = flags.FLAGS

# Used for splitting data into different proportions
def split_data(proportion, data, labels):

    # Find number of instances
    num_examples = data.shape[0]
    # Create index to split on
    split_idx = int(proportion * num_examples)
    
    # Split data into validation and optimization pieces
    data_1, data_2 = data[:split_idx], data[split_idx:]
    # Split labels into validation and optimization pieces
    labels_1, labels_2 = labels[:split_idx], labels[split_idx:]
    
    # Return data as valid_images,valid_labels,opt_images,opt_labels
    return data_1, labels_1, data_2, labels_2

def main(argv):
    # load data
    train_images = np.load(FLAGS.data_dir + 'mnist_train_images.npy')
    train_labels = np.load(FLAGS.data_dir + 'mnist_train_labels.npy')
    test_images = np.load(FLAGS.data_dir + 'mnist_test_images.npy')
    test_labels = np.load(FLAGS.data_dir + 'mnist_test_labels.npy')
    
    # Split train set into optimization set and validation set
    # Shuffle train_images and train_labels before splitting
    idx = np.random.permutation(train_images.shape[0])
    train_images,train_labels = train_images[idx], train_labels[idx]
    valid_images,valid_labels,opt_images,opt_labels = split_data(0.20,train_images,train_labels)
    
    # Remove original train_images and train_labels from memory
    train_images = None
    train_labels = None
    
    # Get number of images in each of the 3 sets
    opt_num_examples = opt_images.shape[0]
    valid_num_examples = valid_images.shape[0]
    test_num_examples = test_images.shape[0]
    
    print('Number of optimization images are %i' %opt_num_examples)
    print('Number of validation images are %i' %valid_num_examples)
    print('Number of testing images are %i' %test_num_examples)
    print('')
    
    # specify the network
    x = tf.placeholder(tf.float32, [None, 784], name='data')
    y = tf.placeholder(tf.float32, [None, 10], name='label')
    with tf.name_scope('linear_model') as scope:
        # L2 regularization has been added to this layer
        hidden = tf.layers.dense(x, 400, activation=tf.nn.relu, name='hidden_layer',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01))
        # L2 regularization has been added to this layer
        output = tf.layers.dense(hidden, 10, name='output_layer',
                                kernel_regularizer=tf.contrib.layers.l2_regularizer(scale=0.01))
        tf.identity(output, name='model_output')

    # define classification loss
    cross_entropy  = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=output)
    
    # collect the regularization losses
    regularization_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    # this value is what we'll pass to `minimize`
    xentropy_w_reg = cross_entropy + 0.1 * sum(regularization_losses)
    
    confusion_matrix_op = tf.confusion_matrix(tf.argmax(y, axis=1), tf.argmax(output, axis=1), num_classes=10)


    # set up training and saving functionality
    global_step_tensor = tf.get_variable('global_step', trainable=False, shape=[], initializer=tf.zeros_initializer)
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(xentropy_w_reg, global_step=global_step_tensor)
    saver = tf.train.Saver()
    
    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        
        # Location to save sessions
        save_directory = './hackathon3_sessions'
        
        # For early stopping
        best_cost=10**20 # Initialize so immediately replaced
        early_stop_epoch_num = 5 # Number of epochs to see no improvement before deciding to stop

        # run training
        batch_size = FLAGS.batch_size
        ########################################################################
        print('Beginning calculations...')
        for epoch in range(FLAGS.max_epoch_num):
    
            #### Go through optimization images for this epoch ####
            # run gradient steps and report mean loss on train data
            ce_vals = []
            tot_opt_minibatch = opt_num_examples // batch_size
            for i in range(tot_opt_minibatch):
                batch_xs = opt_images[i*batch_size:(i+1)*batch_size, :]
                batch_ys = opt_labels[i*batch_size:(i+1)*batch_size, :]       
                _, opt_ce = session.run([train_op, tf.reduce_mean(xentropy_w_reg)], {x: batch_xs, y: batch_ys})
                ce_vals.append(opt_ce)
            avg_opt_ce = sum(ce_vals) / len(ce_vals)
            print('For Epoch %i The OPTIMIZATION CROSS ENTROPY Is :' %epoch + str(avg_opt_ce))
            #############################

            #### Go through validation images for this epoch ####
            # Calculate validation loss
            ce_vals = []
            for i in range(valid_num_examples // batch_size):
                batch_xs = valid_images[i*batch_size:(i+1)*batch_size, :]
                batch_ys = valid_labels[i*batch_size:(i+1)*batch_size, :]
                valid_ce, _ = session.run([tf.reduce_mean(cross_entropy), confusion_matrix_op], {x: batch_xs, y: batch_ys})
                ce_vals.append(valid_ce)
            avg_valid_ce = sum(ce_vals) / len(ce_vals)
            print('For Epoch %i The VALIDATION CROSS ENTROPY Is :' %epoch + str(avg_valid_ce))
            print('')
            ##############################
            
            #### Initiate early stopping check/update ####
            if avg_valid_ce < best_cost:
                best_epoch = epoch
                # Save session
                saved_session = saver.save(session, os.path.join(save_directory, "fashion_mnist_classification"))
                best_cost = avg_valid_ce
                last_improvement = 0
            else:
                last_improvement += 1
                
            # Checks if we have not seen improvement for required number of epochs
            if last_improvement >= early_stop_epoch_num:
                print("No improvement found during last iterations, stopping optimization...")
                saver.restore(session, saved_session) # restore session with the best cost
                # Break out from the loop.
                break
            #####################################
        ########################################################################
                
        # After breaking out of loop assess model using the set aside test set
        
        # Calculate test loss
        print('Processing test set')
        ce_vals = []
        conf_mxs = []
        for i in range(test_num_examples // batch_size):
            batch_xs = test_images[i*batch_size:(i+1)*batch_size, :]
            batch_ys = test_labels[i*batch_size:(i+1)*batch_size, :]
            test_ce, conf_matrix = session.run([tf.reduce_mean(xentropy_w_reg), confusion_matrix_op], {x: batch_xs, y: batch_ys})
            ce_vals.append(test_ce)
            conf_mxs.append(conf_matrix)
        avg_test_ce = sum(ce_vals) / len(ce_vals)
        print('TEST CROSS ENTROPY: ' + str(avg_test_ce))
        print('TEST CONFUSION MATRIX:')
        print(str(sum(conf_mxs)))
        print('')
        print('The epoch we stopped at was %i' %best_epoch)
        
        # Save session at end before final exit
        path_prefix = saver.save(session, os.path.join(save_directory, "fashion_mnist_classification"))
        
if __name__ == "__main__":
    tf.app.run()


Number of optimization images are 44000
Number of validation images are 11000
Number of testing images are 10000



W0917 23:03:23.622506 47439958921344 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0917 23:03:23.624896 47439958921344 deprecation.py:323] From <ipython-input-1-9c799e8e1b3c>:61: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.
W0917 23:03:23.631040 47439958921344 deprecation.py:506] From /util/opt/anaconda/deployed-conda-envs/packages/tensorflow/envs/tensorflow-1.14.0-py36/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be 

Beginning calculations...
For Epoch 0 The OPTIMIZATION CROSS ENTROPY Is :0.4176049399274317
For Epoch 0 The VALIDATION CROSS ENTROPY Is :0.172745247523893

For Epoch 1 The OPTIMIZATION CROSS ENTROPY Is :0.2502139800143513
For Epoch 1 The VALIDATION CROSS ENTROPY Is :0.14099764555523342

For Epoch 2 The OPTIMIZATION CROSS ENTROPY Is :0.22001555049791932
For Epoch 2 The VALIDATION CROSS ENTROPY Is :0.12625104618174107

For Epoch 3 The OPTIMIZATION CROSS ENTROPY Is :0.2082852795550769
For Epoch 3 The VALIDATION CROSS ENTROPY Is :0.11898892547257922

For Epoch 4 The OPTIMIZATION CROSS ENTROPY Is :0.20257842850617386
For Epoch 4 The VALIDATION CROSS ENTROPY Is :0.1155596684325825

For Epoch 5 The OPTIMIZATION CROSS ENTROPY Is :0.19892363402653823
For Epoch 5 The VALIDATION CROSS ENTROPY Is :0.11201322635805065

For Epoch 6 The OPTIMIZATION CROSS ENTROPY Is :0.19553012927486138
For Epoch 6 The VALIDATION CROSS ENTROPY Is :0.1097614425657825

For Epoch 7 The OPTIMIZATION CROSS ENTROPY Is :0.1